In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster
@pyimport sklearn.metrics as metrics

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure10"

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

In [ ]:
data_info_dict

In [ ]:
analyzer = "chuyu"

In [ ]:
function neighbor_percent(corr_map_pairwise_all_1, corr_map_pairwise_all_2, neighbor_percent_list)
    
    percent_stay_all_n = fill(NaN32, length(neighbor_percent_list), size(corr_map_pairwise_all_1, 1))
    for which_neuron = 1:size(corr_map_pairwise_all_1, 1)
        neighbor_rank_1 = reverse(sortperm(corr_map_pairwise_all_1[which_neuron,:]))
        neighbor_rank_2 = reverse(sortperm(corr_map_pairwise_all_2[which_neuron,:]))
        
        valid_neighbors_1 = findall(corr_map_pairwise_all_1[which_neuron,:].>=-1)
        valid_neighbors_2 = findall(corr_map_pairwise_all_2[which_neuron,:].>=-1)
        
        n_neighbors_list = floor.(Int32, neighbor_percent_list.*length(valid_neighbors_1))
        
        for (i_neighbor, n_neighbors) in enumerate(n_neighbors_list)
            top_neighbors_1 = intersect(neighbor_rank_1[1:n_neighbors], valid_neighbors_1)
            top_neighbors_2 = intersect(neighbor_rank_2[1:n_neighbors], valid_neighbors_2)
            
            percent_stay = sum(whether_in(top_neighbors_2,  top_neighbors_1))/length(top_neighbors_1)
            percent_stay_all_n[i_neighbor,which_neuron] = percent_stay
        end
    end 
    return percent_stay_all_n
end

# Save figure

In [ ]:
key = "corner_cue_circle"

data_info_all = data_info_dict[key]


n_bins = 31

trials_mean_tog = []
earlylate_mean_tog = []
    

for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]
    
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    cluster_file = h5open(joinpath(data_path(ds_save_cy_1), "cluster_analysis_neighbors.h5"))
    edge_neuron_index = read(cluster_file,"edge_neuron_index")
    center_neuron_index = read(cluster_file,"center_neuron_index")
    percent_stay_all_n_mean_edge = read(cluster_file,"percent_stay_all_n_mean_edge")
    percent_stay_all_n_mean_earlylate_edge = read(cluster_file,"percent_stay_all_n_mean_earlylate_edge")
    percent_stay_all_n_mean_center = read(cluster_file,"percent_stay_all_n_mean_center")
    percent_stay_all_n_mean_earlylate_center = read(cluster_file,"percent_stay_all_n_mean_earlylate_center")
    percent_stay_all_n_mean_earlylate_percent = read(cluster_file,"percent_stay_all_n_mean_earlylate_percent")
    percent_stay_all_n_mean_percent = read(cluster_file,"percent_stay_all_n_mean_percent")
    close(cluster_file)    
        
        
    println(sum(edge_neuron_index))
    println(sum(center_neuron_index))
    save_file_name = "compare_map_results.h5"
    compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(compare_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    close(file)
    
    
    fig = figure(figsize=(0.75,0.75))
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_percent, label= "S1 early vs late", color = "k")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_edge, label= "S1 early vs late (edge cells)", color = "k", linestyle="dashed")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_center, label= "S1 early vs late (center cells)", color = "k", linestyle="dotted")
    
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_percent, label= "S1 vs S2", color = "r")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_edge, label= "S1 vs S2 (edge cells)", color = "r", linestyle="dashed")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_center, label= "S1 vs S2 (center cells)", color = "r", linestyle="dotted")
    
    plot([0,100],[0,100], color="gray", label="Random")
    axis("equal")
    
    if which_data == 1
        xticks([])
        xlabel("")
        ylabel("Neighbor \n retention %", labelpad = -1)
        yticks([0, 50, 100]) 
    end
    if which_data == 2
        xticks([])
        xlabel("")
        ylabel("Neighbor \n retention %", labelpad = -1)
        yticks([0, 50, 100]) 
    end
    if which_data == 3
        xticks([])
        xlabel("")

        ylabel("Neighbor \n retention %", labelpad = -1)
        yticks([0, 50, 100]) 
    end
    if which_data == 4
        xticks([0, 50, 100])
        xlabel("Neighbor %")

        ylabel("Neighbor \n retention %", labelpad = -1)
        yticks([0, 50, 100]) 
    end
    
    # title(experiment_filename_1)
    fig.savefig(joinpath(fig_path, "staying_neighbors_v3_$(which_data).pdf"), bbox_inches="tight",transparent = true);
end

# Save source data

In [ ]:
data_dict = Dict()
data_dict["Neighbor %"] = 100 .*numpy.linspace(0.02, 1, 50)

key = "corner_cue_circle"

data_info_all = data_info_dict[key]


n_bins = 31

trials_mean_tog = []
earlylate_mean_tog = []
    

for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]
    
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    cluster_file = h5open(joinpath(data_path(ds_save_cy_1), "cluster_analysis_neighbors.h5"))
    edge_neuron_index = read(cluster_file,"edge_neuron_index")
    center_neuron_index = read(cluster_file,"center_neuron_index")
    percent_stay_all_n_mean_edge = read(cluster_file,"percent_stay_all_n_mean_edge")
    percent_stay_all_n_mean_earlylate_edge = read(cluster_file,"percent_stay_all_n_mean_earlylate_edge")
    percent_stay_all_n_mean_center = read(cluster_file,"percent_stay_all_n_mean_center")
    percent_stay_all_n_mean_earlylate_center = read(cluster_file,"percent_stay_all_n_mean_earlylate_center")
    percent_stay_all_n_mean_earlylate_percent = read(cluster_file,"percent_stay_all_n_mean_earlylate_percent")
    percent_stay_all_n_mean_percent = read(cluster_file,"percent_stay_all_n_mean_percent")
    close(cluster_file)    
        
        
    println(sum(edge_neuron_index))
    println(sum(center_neuron_index))
    save_file_name = "compare_map_results.h5"
    compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(compare_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    close(file)
    
    
    fig = figure(figsize=(1,1))
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_percent, label= "S1 early vs late", color = "gray")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_edge, label= "S1 early vs late (edge cells)", color = "gray", linestyle="dashed")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_earlylate_center, label= "S1 early vs late (center cells)", color = "gray", linestyle="dotted")
    
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_percent, label= "S1 vs S2", color = "r")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_edge, label= "S1 vs S2 (edge cells)", color = "r", linestyle="dashed")
    plot(100 .*numpy.linspace(0.02, 1, 50), 100 .*percent_stay_all_n_mean_center, label= "S1 vs S2 (center cells)", color = "r", linestyle="dotted")
    
    data_dict["(Fish $(which_data)) Neighbor retention % S1 early vs late"] = 100 .*percent_stay_all_n_mean_earlylate_percent
    data_dict["(Fish $(which_data)) Neighbor retention % S1 early vs late (edge cells)"] = 100 .*percent_stay_all_n_mean_earlylate_edge
    data_dict["(Fish $(which_data)) Neighbor retention % S1 early vs late (center cells)"] = 100 .*percent_stay_all_n_mean_earlylate_center
    data_dict["(Fish $(which_data)) Neighbor retention % S1 vs S2"] = 100 .*percent_stay_all_n_mean_percent
    data_dict["(Fish $(which_data)) Neighbor retention % S1 vs S2 (edge cells)"] = 100 .*percent_stay_all_n_mean_edge
    data_dict["(Fish $(which_data)) Neighbor retention % S1 vs S2 (center cells)"] = 100 .*percent_stay_all_n_mean_center
end

In [ ]:
@pyimport pandas as pd


function fill_length(vec, desired_length)
    [vec; fill(NaN, desired_length-length(vec))]
end

In [ ]:
max_length = maximum([length(data_dict[key]) for key in keys(data_dict)])

for key in keys(data_dict)
    data_dict[key] = fill_length(data_dict[key], max_length)
end

df = pd.DataFrame.from_dict(data_dict)
df = df.sort_index(axis=1)

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Source Data Fig. 6.xlsx"
sheet_name = "Fig. 6e"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a", if_sheet_exists="replace")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end